In [18]:
from pyomo.environ import *

In [19]:
a = {'Seattle': 350, 'San Diego': 600}
d = {'New York': 325, 'Chicago': 300, 'Topeka': 275}
c = {
     ('Seattle', 'New York'): 2.5,  
     ('Seattle', 'Chicago'): 1.7, 
     ('Seattle', 'Topeka'): 1.8,
     ('San Diego', 'New York'): 2.5,  
     ('San Diego', 'Chicago'): 1.8, 
     ('San Diego', 'Topeka'): 1.4
    }

In [ ]:
plants = list(a.keys())
markets = list(d.keys())

model = ConcreteModel()
model.x = Var( plants, markets, within=NonNegativeReals )

model.cost = Objective(
    expr = sum( c[i,j]*model.x[i,j] for i in plants for j in markets ),
    sense = minimize )

model.supply = ConstraintList()
for i in plants:
    model.supply.add(
        sum( model.x[i,j] for j in markets ) <= a[i] )
    
model.demand = ConstraintList()
for j in markets:
    model.demand.add(
        sum( model.x[i,j] for i in plants ) >= d[j] )
    
#model.pprint()

In [20]:
solver = SolverFactory('glpk')
solver.solve(model)
model.display()

Model unknown

  Variables:
    x : Size=6, Index=x_index
        Key                       : Lower : Value : Upper : Fixed : Stale : Domain
         ('San Diego', 'Chicago') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('San Diego', 'New York') :     0 : 325.0 :  None : False : False : NonNegativeReals
          ('San Diego', 'Topeka') :     0 : 275.0 :  None : False : False : NonNegativeReals
           ('Seattle', 'Chicago') :     0 : 300.0 :  None : False : False : NonNegativeReals
          ('Seattle', 'New York') :     0 :   0.0 :  None : False : False : NonNegativeReals
            ('Seattle', 'Topeka') :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1707.5

  Constraints:
    supply : Size=2
        Key : Lower : Body  : Upper
          1 :  None : 300.0 : 350.0
          2 :  None : 600.0 : 600.0
    demand : Size=3
        Key : Lowe

In [21]:
print("Objective : ", model.cost())

Objective :  1707.5
